In [24]:
import os, csv
import pandas as pd
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [25]:
# convert all txt files into a single csv file
corpus = './corpus.csv'
test_corpus = './test_corpus.csv'
train_path = './train-mails'
test_path = './test-mails'
with open(corpus, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['File Name', 'Content', 'Spam or Valid'])

    for txt_filename in os.listdir(train_path):
        if txt_filename.endswith('.txt'):
            with open(os.path.join(train_path, txt_filename), 'r', encoding='utf-8') as txt_file:
                content = txt_file.read()
                spam_or_valid = 'spam' if txt_filename.startswith('spmsg') else 'valid'
                csv_writer.writerow([txt_filename, content, spam_or_valid])

with open(test_corpus, 'w', newline='', encoding='utf-8') as test_csv_file:
    csv_writer = csv.writer(test_csv_file)
    csv_writer.writerow(['File Name', 'Content', 'Spam or Valid'])

    for txt_filename in os.listdir(test_path):
        if txt_filename.endswith('.txt'):
            with open(os.path.join(test_path, txt_filename), 'r', encoding='utf-8') as txt_file:
                content = txt_file.read()
                spam_or_valid = 'spam' if txt_filename.startswith('spmsg') else 'valid'
                csv_writer.writerow([txt_filename, content, spam_or_valid])

In [26]:
# Create a dictionary of words (dropping all non-words like punctuation characters, single characters)
# choosing the 2000 most frequent words from training set
csv_file = pd.read_csv('./corpus.csv', encoding='utf-8')
test_csv_file = pd.read_csv('./test_corpus.csv', encoding='utf-8')
csv_file['Content'] = csv_file['Content'].str.replace(r'[^\w\s]+|(?<=\s)\S(?=\s)|\S(?<=\s)','',regex=True)
test_csv_file['Content'] = csv_file['Content'].str.replace(r'[^\w\s]+|(?<=\s)\S(?=\s)|\S(?<=\s)','',regex=True)
dictionary = {}
for i in range(len(csv_file['Content'])):
    for word in csv_file['Content'][i].split():
        if word not in dictionary:
            if len(word) > 1:
                dictionary[word] = 1
        elif word in dictionary:
            dictionary[word] += 1
    del dictionary['Subject']
    

top_2000_words = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)[:2000]
sortedDictionary = dict(top_2000_words)
#for word in top_2000_words:
#   print(word)

('20', 1764)
('email', 1664)
('order', 1414)
('address', 1299)
('report', 1217)
('mail', 1133)
('language', 1099)
('send', 1080)
('program', 1009)
('our', 991)
('list', 946)
('one', 921)
('name', 883)
('receive', 826)
('free', 801)
('money', 797)
('nt', 759)
('work', 756)
('3d', 736)
('information', 684)
('business', 669)
('please', 657)
('university', 600)
('us', 567)
('day', 559)
('10', 550)
('follow', 545)
('internet', 533)
('over', 514)
('call', 488)
('http', 479)
('check', 475)
('each', 466)
('linguistic', 460)
('include', 452)
('com', 450)
('100', 442)
('want', 426)
('need', 426)
('number', 424)
('letter', 420)
('many', 412)
('here', 400)
('market', 398)
('start', 390)
('even', 388)
('fax', 384)
('form', 381)
('most', 377)
('first', 374)
('web', 372)
('50', 367)
('service', 365)
('interest', 364)
('software', 362)
('read', 352)
('remove', 349)
('those', 346)
('week', 346)
('credit', 334)
('every', 333)
('site', 331)
('ll', 327)
('english', 325)
('edu', 319)
('much', 318)
('produc

In [27]:
# print(csv_file['Content'][0])

Subject re    882    np np

 deat  sun  15 dec 91   25   est   michael  mmorse  vm1  yorku  ca   subject  re    864 query   wlodek zadrozny ask  anything interest   construction    np np     second   much relate  consider construction form  discuss list late reduplication   logical sense  john mcnamara name  tautologous thus   level  indistinguishable    here     john mcnamara name  tautologous support those logicbase semantics irrelevant natural language  sense tautologous  supplies value attribute follow attribute value  fact value nameattribute relevant entity  chaim shmendrik    john mcnamara name  false  tautology    reduplication  either  


In [28]:
# Transform each mail into a word count vector basing on the dictionary of words
#word_count_matrix = CountVectorizer(vocabulary=sortedDictionary.keys()).transform(csv_file['Content'])

X_train = []
y_train = []
X_test = []
y_test = []

for mail_index in range(len(csv_file['Content'])):
    X_train.append([])
    for key in sortedDictionary.keys():
        word_counter = 0
        for word in csv_file['Content'][mail_index].split():
            if word == key:
                word_counter += 1
        X_train[-1].append(word_counter)
    y_train.append(csv_file['Spam or Valid'][mail_index])

for mail_index in range(len(test_csv_file['Content'])):
    X_test.append([])
    for key in sortedDictionary.keys():
        word_counter = 0
        for word in test_csv_file['Content'][mail_index].split():
            if word == key:
                word_counter += 1
        X_test[-1].append(word_counter)
    y_test.append(csv_file['Spam or Valid'][mail_index])

print('Matrix dimensions:', len(X_train), 'x', len(X_train[0]))

Matrix dimensions: 702 x 2000


In [29]:
print(len(X_train))

702


In [30]:
# Train a Bayes classifier (MultinomialNB from sklearn.naive_bayes)
# Check with the test set the classifier

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)
clf = MultinomialNB()
clf.fit(X_train, y_train)
print('Accuracy:', clf.score(X_test, y_test))

#TODO: correct train and test code

Accuracy: 0.9929078014184397
